# Fuzzy Search 

## PyData DC 2016

Jiaqi Liu

https://github.com/jiaqi216/pydata_dc


    

**whoami**

- Jiaqi Liu
- Data Scientist @ Capital One
- NYC based
- work mostly at the intersection of data science and engineering
- https://github.com/jiaqi216/pydata_dc


**Fuzzy Match Algorithms**

- Soundex 
- Levenshtein
- n-gram (model?)
- NLTK/Word2Vec

**Soundex**

- phonetic algorithm
- index by sound as pronounced in English
- assigns a soundex coding 
- ideal for spelling inconsistencies

**American Soundex Coding**

[http://www.archives.gov/research/census/soundex.html](http://www.archives.gov/research/census/soundex.html)

every soundex code is a letter and three numbers

| Number | Letter |
|---------|----------|
|1|B,F,P,V|
|2|C,G,J,K,Q,S,X,Z|
|3|D,T|
|4|L|
|5|M,N|
|6|R|

Ignore A,E,I,O,U,H,W,Y

In [46]:
import jellyfish as j

a=j.soundex('WASHINGTON')
print(a)
b=j.soundex('WUSHINGTON')
print(b)

W252
W252


In [8]:
a=j.soundex('LGA')
print(a)
b=j.soundex('LAGUARDIA')
print(b)

L200
L263


In [109]:
a=j.soundex('SBUX#123')
print(a)
a2=j.soundex('SBUX')
print(a2)

S120
S120


**Soundex**
- Soundex is pretty easy to implement
- Computationally fast
- only works on ASCII characters (no foreign languages)
- How do you calculate distance

**Levenshtein distance**
- also call edit distance
- accounts for how many characters you have to change to have the same string
- computationally fast (can handle real time processing)
- pairwise comparison

In [110]:
import Levenshtein as l

l.distance('SMYTHE', 'SMITH')

2

In [111]:
l.distance('SBUX', 'Starbucks')

8

In [112]:
a=j.soundex('SBUX#123')
print(a)
b=j.soundex('Starbucks')
print(b)

l.distance(a,b)

S120
S361


3

Pitfall: Comparing Addresses

In [113]:
str99 = '99 Broadway'
str100 = '100 Broadway'
str999 = '999 Broadway'

l.distance(str99, str100)

3

- weighing numbers differently from letters

In [116]:
l.distance('café', 'cafe')

1

In [117]:
l.distance('cafe', 'cafe')

0

Longer Strings

In [4]:
str1='Hello world, python is great'
str2='Hello world, python is good'
l.distance(str1,str2)

4

In [5]:
import Levenshtein as l
str1='great'
str2='good'
l.distance(str1,str2)

4

- counting raw edits penalizes long strings: use a ratio of edits to length


**Damerau-Levenshtein**
- like Levenshtein but accounts for transposition of adjacent characters

In [9]:
j.damerau_levenshtein_distance('recieve', 'receive')

1

In [10]:
j.levenshtein_distance('recieve', 'receive')

2

In [44]:
import csv
def load_cities_list():
    cities = []
    with open('data/misspelled_cities.csv') as data_file:
        reader = csv.reader(data_file, delimiter='|')
        for correct, wrong in reader:
            cities.append({'correct': correct,'wrong': wrong})
    return cities

In [47]:
for city in load_cities_list():
    print(j.levenshtein_distance(city['correct'], city['wrong']))

9
1
1
2
2
1
2
2
1
2
1
4
11
1
2
12
2
1
5
1
1
2
2
1
1
1
1
1
1
1
2
1
1
1
1
1
1
3
10


**n-gram/Trigram**

- groupings of letters (takes into more context)
- proper unit of analysis (1-gram, 2-gram, 3-gram)
- slower to implement - need to calculate n-gram for each string

Sentences

In [120]:
def ngram(tokens, n):
    grams =[tokens[i:i+n] for i in range(len(tokens)-(n-1))]
    return grams

In [122]:
sentence_gram = "The quick brown fox jumped over a lazy dog".split()
grams = ngram(sentence_gram, 3)

for gram in grams:
    print(gram)


['The', 'quick', 'brown']
['quick', 'brown', 'fox']
['brown', 'fox', 'jumped']
['fox', 'jumped', 'over']
['jumped', 'over', 'a']
['over', 'a', 'lazy']
['a', 'lazy', 'dog']


Words

In [123]:
def ngram(tokens, n):
    grams =[tokens[i:i+n] for i in range(len(tokens)-(n-1))]
    return grams

In [128]:
word_gram = "Starbucks"
grams = ngram(word_gram, 3)

for gram in grams:
    print(gram)


Sta
tar
arb
rbu
buc
uck
cks


Scoring Similarity: Jaccard similarity

In [126]:
def get_sim(a_tri,b_tri):
    intersect = len(set(a_tri) & set(b_tri))
    union = len(set(a_tri) | set(b_tri))
    return float(intersect)/(union)

In [129]:
print(grams)
get_sim(grams, grams)

['Sta', 'tar', 'arb', 'rbu', 'buc', 'uck', 'cks']


1.0

Word2Vec: uses cosine distance
cosine distance between two vectors is nice but like have to find a way to quantify

In [130]:
new_gram = ngram('Starbux', 3)
get_sim(grams, new_gram)

0.5

In [132]:
a_gram = ngram('RichmondConventionCtner', 3)
b_gram = ngram('Richmond Convention Center', 3)
get_sim(a_gram, b_gram)

0.4666666666666667

In [133]:
a_gram = ngram('receved data', 3)
b_gram = ngram('received date', 3)
get_sim(a_gram, b_gram)

0.5

Trigam Search with Postgres

In [1]:
from sqlalchemy import create_engine

engine = create_engine('postgresql://hhu373@localhost/cities')
connection = engine.connect()

In [17]:
#https://www.postgresql.org/docs/9.1/static/pgtrgm.html
query="""
    SELECT 
                a.alt_spelling, 
                similarity(lower(a.alt_spelling), :city) as similarity 
    FROM        fuzzy_names as a 
    WHERE       lower(a.alt_spelling) % :city
    ORDER BY    similarity DESC"""

In [18]:
from sqlalchemy.sql import text

city_name = 'washington'
res = engine.execute(text(query), city=city_name)

In [19]:
res.fetchall()

[(' Washington D.C.', 0.733333),
 (' Washinton DC', 0.5),
 (' WSHINGTON DC', 0.5),
 (' WSHINGTONDC', 0.4375)]

**Other Similarity Metrics**
- NLTK: wordnet
- Word2Vec: uses cosine distance
    - cosine distance between two vectors


In [20]:
from nltk.corpus import wordnet


In [21]:
word1 = wordnet.synsets("blue")
word2 = wordnet.synsets("green")
word1[0].wup_similarity(word2[0])

0.875